In [5]:
import pandas as pd
import numpy as np
from gurobipy import Model, GRB, quicksum

In [136]:
#data
inc_init = pd.read_csv("/Users/sabrina/Desktop/MGT 189/IEOR-4004-Project-1/ChildCareDeserts_Data/avg_individual_income.csv")
fac_init = pd.read_csv("/Users/sabrina/Desktop/MGT 189/IEOR-4004-Project-1/ChildCareDeserts_Data/child_care_regulated.csv")
emp_init = pd.read_csv("/Users/sabrina/Desktop/MGT 189/IEOR-4004-Project-1/ChildCareDeserts_Data/employment_rate.csv")
pop_init = pd.read_csv("/Users/sabrina/Desktop/MGT 189/IEOR-4004-Project-1/ChildCareDeserts_Data/population.csv")
loc= pd.read_csv("/Users/sabrina/Desktop/MGT 189/IEOR-4004-Project-1/ChildCareDeserts_Data/potential_locations.csv")

fac_init.rename(columns={'zip_code': 'zipcode'}, inplace=True)
inc_init.rename(columns={'ZIP code': 'zipcode'}, inplace=True)

#clean zipcode
def clean_zipcode_column(df, column_name):
    """
    """
    #convert zipcode to string and take the first 5 characters
    df.loc[:, column_name] = df[column_name].astype(str).str.slice(0, 5)
    return df

fac_init = clean_zipcode_column(fac_init, 'zipcode')
pop_init = clean_zipcode_column(pop_init, 'zipcode')
emp_init = clean_zipcode_column(emp_init, 'zipcode')
inc_init = clean_zipcode_column(inc_init, 'zipcode')

Creating new columns: Capacity for age 0-5 and capacity for age 5-12 by summing existing age group data. Fill with 0 capacity if data has N/A entry

In [137]:
fac_init.rename(columns={'zip_code': 'zipcode'}, inplace=True)
fac_init['cap_0_5'] = (
    fac_init['infant_capacity'].fillna(0) +
    fac_init['toddler_capacity'].fillna(0) +
    fac_init['preschool_capacity'].fillna(0)
)
fac_init['cap_5_12'] = fac_init['school_age_capacity'].fillna(0)
fac_init['total_cap'] = fac_init['cap_0_5'] + fac_init['cap_5_12']


Creating new columns: Population by age group 0-5 and 5-12

In [138]:
# --- Define child population columns manually ---
# Replace NaNs with 0 in case some ZIPs are missing age data
pop_init = pop_init.fillna(0)

# 0–5 = '-5' (since that's your label)
pop_init['pop_0_5'] = pop_init['-5']

# 5–12 = '5-9' + part of '10-14' (approximation)
# We'll assume 3/5 of the 10–14 group are aged 10–12 (since 3 out of 5 years)
pop_init['pop_5_12'] = pop_init['5-9'] + 0.6 * pop_init['10-14']

# total children 0–12
pop_init['pop_children'] = pop_init['pop_0_5'] + pop_init['pop_5_12']

Longitude into 

Align datsets
- here we introduce $X := \text{total number of exisiting childcare slots in}\ z$
- since facility location data are on a facility level, we groupby zipcode to align with other zipcode level data

In [139]:
#zipcode level
fac_zip = (
    fac_init
    .groupby('zipcode')[['cap_0_5', 'cap_5_12', 'total_cap']]
    .sum()
    .reset_index()
    .rename(columns={'total_cap': 'X'})
)

#reindex and fill with mean
all_zips = sorted(set(emp_init['zipcode']) | set(inc_init['zipcode']) | set(pop_init['zipcode']) | set(fac_zip['zipcode']))

def reindex_and_fill(df, zip_col, value_cols):
    df = df.set_index(zip_col).reindex(all_zips)
    for c in value_cols:
        df[c] = df[c].fillna(df[c].mean())  # fill missing with column mean
    df = df.reset_index().rename(columns={'index': 'zipcode'})
    return df
emp = reindex_and_fill(emp_init, 'zipcode', ['employment rate'])
inc = reindex_and_fill(inc_init, 'zipcode', ['average income'])
pop = reindex_and_fill(pop_init, 'zipcode', ['pop_0_5', 'pop_5_12', 'pop_children'])
fac = fac_zip.set_index('zipcode').reindex(all_zips).fillna(0).reset_index()

for df in [pop, fac, loc, emp, inc]:
    df['zipcode'] = df['zipcode'].astype(str).str.zfill(5)



Check missing value and shape

In [140]:

merged = (
    pop.merge(emp, on='zipcode', how='left')
            .merge(inc, on='zipcode', how='left')
            .merge(fac, on='zipcode', how='left')
)

merged[['cap_0_5', 'cap_5_12', 'X']] = merged[['cap_0_5', 'cap_5_12', 'X']].fillna(0)



merged_clean = merged[[
    'zipcode',
    'pop_0_5',
    'pop_5_12',
    'pop_children',
    'employment rate',
    'average income',
    'X',
    'cap_0_5',
    'cap_5_12'
]]

print("Remaining missing values:\n", merged_clean.isna().sum())

# --- Quick sanity check ---
print("\nExample rows:")
display(merged_clean.head(10))

Remaining missing values:
 zipcode            0
pop_0_5            0
pop_5_12           0
pop_children       0
employment rate    0
average income     0
X                  0
cap_0_5            0
cap_5_12           0
dtype: int64

Example rows:


,zipcode,pop_0_5,pop_5_12,pop_children,employment rate,average income,X,cap_0_5,cap_5_12
0,10001,744.0,1349.2,2093.2,0.595097,102878.033603,585.0,0.0,585.0
1,10002,2142.0,4964.8,7106.8,0.520662,59604.041165,4526.0,18.0,4508.0
2,10003,1440.0,1605.8,3045.8,0.497244,114273.049645,1995.0,0.0,1995.0
3,10004,433.0,278.6,711.6,0.506661,132004.310345,263.0,0.0,263.0
4,10005,484.0,341.4,825.4,0.665833,121437.713311,39.0,0.0,39.0
5,10006,128.0,141.0,269.0,0.631692,126377.118644,156.0,14.0,142.0
6,10007,605.0,625.0,1230.0,0.528910,138853.904282,284.0,0.0,284.0
7,10009,1896.0,2759.0,4655.0,0.514567,77133.233533,1678.0,18.0,1660.0
8,10010,1422.0,2160.8,3582.8,0.492749,116272.698810,234.0,0.0,234.0
9,10011,1209.0,2018.4,3227.4,0.557000,120420.792079,1950.0,42.0,1908.0


$Z:$ set of zipcodes 

$F_z:$ set of existing facilities in zip code $z$

$L_z:$ set of potential facility locations in zip code $z$


In [154]:
# --- Configuration ---
facility_sizes = {'small': 100, 'medium': 200, 'large': 400}
facility_costs = {'small': 65000, 'medium': 95000, 'large': 115000}

# --- Helper datasets ---
zipcodes = pop['zipcode'].unique()
fac_init['facility_id'] = fac_init['facility_id'].astype(str)

results, objectives = {}, {}

# --- Loop over ZIPs ---
for z in zipcodes:
    m = gp.Model(f"zip_{z}")
    m.Params.OutputFlag = 0   # silence Gurobi output

    # Facilities and potential locations in this ZIP
    fz = fac_init[fac_init['zipcode'] == z]
    lz = loc[loc['zipcode'] == z]

    if fz.empty and lz.empty:
        continue  # skip ZIPs with no facility or potential site

    # --- Decision variables ---
    x = m.addVars(fz['facility_id'], lb=0, name="expand_amount")             # expansion amount
    x_05 = m.addVars(fz['facility_id'], lb=0, name="expand_under5")          # 0–5 expansion amount
    w = m.addVars(fz['facility_id'], vtype=GRB.BINARY, name="expand_indicator")  # binary if expanded

    y = m.addVars(lz.index, facility_sizes.keys(), vtype=GRB.BINARY, name="build")  # new build
    y_new_05 = m.addVars(lz.index, facility_sizes.keys(), lb=0, vtype=GRB.INTEGER,
                         name="build_under5")  # 0–5 capacity from new builds

    # --- Constraints ---
    # Expansion limits
    for f in fz.index:
        fid = fz.loc[f, 'facility_id']
        limit = min(1.2 * fz.loc[f, 'total_cap'], 500)
        m.addConstr(x[fid] <= limit * w[fid], name=f"expansion_limit_{fid}")
        m.addConstr(x_05[fid] <= x[fid], name=f"under5_le_total_{fid}")

    # At most one facility per potential site
    for l in lz.index:
        m.addConstr(quicksum(y[l, s] for s in facility_sizes) <= 1, name=f"one_size_loc_{l}")

    # Capacity linkage for new 0–5 slots
    for l in lz.index:
        for s in facility_sizes:
            m.addConstr(y_new_05[l, s] <= facility_sizes[s] * y[l, s], name=f"under5_new_{l}_{s}")

    # --- Demand and Policy Constraints ---
    N = pop.loc[pop['zipcode'] == z, ['pop_0_5', 'pop_5_12']].sum(axis=1).values[0]
    N_05 = pop.loc[pop['zipcode'] == z, 'pop_0_5'].values[0]
    E = emp.loc[emp['zipcode'] == z, 'employment rate'].values[0]
    I = inc.loc[inc['zipcode'] == z, 'average income'].values[0]

    high_demand = (E >= 0.6) or (I <= 60000)
    threshold = 0.5 if high_demand else 1/3

    # Total available slots in this ZIP
    A_total = (
        quicksum(fz.loc[f, 'total_cap'] + x[fz.loc[f, 'facility_id']] for f in fz.index)
        + quicksum(facility_sizes[s] * y[l, s] for l in lz.index for s in facility_sizes)
    )
    m.addConstr(A_total >= threshold * N, name=f"demand_threshold_{z}")

    # 0–5 slots requirement
    A_05 = (
        quicksum(fz.loc[f, 'cap_0_5'] + x_05[fz.loc[f, 'facility_id']] for f in fz.index)
        + quicksum(y_new_05[l, s] for l in lz.index for s in facility_sizes)
    )
    m.addConstr(A_05 >= (2 / 3) * N_05, name=f"under5_policy_{z}")

    # --- Objective ---
    expansion_cost = quicksum((20000 + 200 * fz.loc[f, 'total_cap']) * w[fz.loc[f, 'facility_id']]
                              + 100 * x_05[fz.loc[f, 'facility_id']]
                              for f in fz.index)

    build_cost = quicksum(facility_costs[s] * y[l, s] + 100 * y_new_05[l, s]
                          for l in lz.index for s in facility_sizes)

    m.setObjective(expansion_cost + build_cost, GRB.MINIMIZE)

    # --- Optimize ---
    m.optimize()
    objectives[z] = m.ObjVal if m.status == GRB.OPTIMAL else None

# --- Save results safely ---
    row = {}
    if m.status == GRB.OPTIMAL or m.status == GRB.SUBOPTIMAL:
        for v in m.getVars():
            try:
                row[v.VarName] = v.X
            except gp.GurobiError:
                row[v.VarName] = None
    else:
        row['Status'] = m.status
    results[z] = row

# === Collect & summarize results ===
res_df = pd.DataFrame(results).T
res_df['Objective'] = pd.Series(objectives)

# --- Filter only feasible ZIPs ---
feasible_zips = [z for z, v in objectives.items() if v is not None]
if not feasible_zips:
    print("⚠️ No feasible ZIP-level models found.")
else:
    total_cost = sum(objectives[z] for z in feasible_zips)
    
    # Extract key metrics
    # Count new facilities (any y[l,s] = 1)
    new_facilities = 0
    added_slots_expansion = 0
    under5_upgraded = 0
    construction_cost = 0
    expansion_cost = 0
    equipment_cost = 0

    for z, row in results.items():
        for name, val in row.items():
            if val == 0 or pd.isna(val):
                continue
            
            # New facilities
            if name.startswith("build[") and val > 0:
                new_facilities += 1
                for s in facility_sizes:
                    if s in name:
                        construction_cost += facility_costs[s] * val

            # Expansion
            elif name.startswith("expand_amount["):
                added_slots_expansion += val
                expansion_cost += 200 * val

            # Equipment (under-5 upgrades)
            elif "expand_under5" in name or "build_under5" in name:
                under5_upgraded += val
                equipment_cost += 100 * val

    # --- Print formatted report ---
    print("\n=== Optimal Solution ===")
    print(f"Total cost: ${total_cost:,.2f}")
    print(f"  Construction: ${construction_cost:,.2f}")
    print(f"  Expansion   : ${expansion_cost:,.2f}")
    print(f"  Equipment   : ${equipment_cost:,.2f}\n")

    print("Decisions:")
    print(f"  Number of New facilities: {int(new_facilities):,}")
    print(f"  Additional Slots: {int(added_slots_expansion):,}")
    print(f"  Additional 0-5 Slots : {int(under5_upgraded):,}")

    # --- Optional: display top few ZIPs by cost ---
    print("\nTop 10 ZIPs by cost:")
    top10 = res_df.loc[feasible_zips, ['Objective']].sort_values('Objective', ascending=False).head(10)
    display(top10)





=== Optimal Solution ===
Total cost: $353,342,107.90
  Construction: $212,430,000.00
  Expansion   : $56,251,040.00
  Equipment   : $64,107,307.90

Decisions:
  New facilities (count): 2,138
  Added slots via expansion: 281,255
  Under-5 slots upgraded: 641,073

Top 10 ZIPs by cost:


,Objective
11219,4.284533e+06
10977,3.006100e+06
11368,2.588867e+06
10950,2.345100e+06
11223,2.307800e+06
11208,2.302733e+06
11220,2.221267e+06
11204,2.166200e+06
10468,2.148333e+06
11206,2.124133e+06


,"build[200,small]","build[200,medium]","build[200,large]","build[201,small]","build[201,medium]","build[201,large]","build[202,small]","build[202,medium]","build[202,large]","build[203,small]",...,"build_under5[215397,small]","build_under5[215397,medium]","build_under5[215397,large]","build_under5[215398,small]","build_under5[215398,medium]","build_under5[215398,large]","build_under5[215399,small]","build_under5[215399,medium]","build_under5[215399,large]",Objective
06390,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65000.0
10001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,279600.0
10002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1177800.0
10003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,441000.0
10004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,143900.0
10005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,212300.0
10006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103600.0
10007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,220400.0
10009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,586400.0
10010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,439800.0


In [155]:
res_df = res_df.round(0).astype(int)
display(res_df.head(20))

,expand_amount[837597],expand_amount[661697],expand_amount[837329],expand_amount[350076],expand_amount[292419],expand_amount[229433],expand_amount[827488],expand_amount[912862],expand_amount[893683],expand_under5[837597],...,"build_under5[297,small]","build_under5[297,medium]","build_under5[297,large]","build_under5[298,small]","build_under5[298,medium]","build_under5[298,large]","build_under5[299,small]","build_under5[299,medium]","build_under5[299,large]",Objective
10001,100.8,0.0,0.0,0.0,94.8,0.0,0.0,0.0,350.4,100.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200600.000000
10002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,476000.000000
10003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,323200.000000
10004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98866.666667
10005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,147300.000000
10006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58333.333333
10007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,178733.333333
10009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,434000.000000
10010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,372800.000000
10011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,244600.000000
